In [61]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer

import torch
from torch.utils.data import Dataset

# 1. Preprocessing


In [62]:
def make_df(data):
    with open(data, "r", encoding='utf-8') as file:
        lines = file.readlines()

    comments = []
    aspects = []
    sentiments = []

    for i in range(1, len(lines), 4):
        comment = lines[i].strip()
        sentiment_line = lines[i + 1].strip()
    
        matches = re.findall(r"\{(.*?),(.*?)\}", sentiment_line)
        list_aspect = []
        list_sentiment = []
        
        for match in matches:
            list_sentiment.append(match[1].strip())  # Extract the sentiment
            list_aspect.append(match[0].strip())    # Extract the aspect

        comments.append(comment)
        sentiments.append(list_sentiment)
        aspects.append(list_aspect)


    # Create a DataFrame from the lists
    df = pd.DataFrame({
        'comment': comments,
        'aspect': aspects,
        'sentiment': sentiments
    })
    return df

In [63]:
train_df = make_df("/kaggle/input/dataset-vlsp2018/VLSP2018-SA-hotel-train (3-2-2018).txt")
val_df = make_df("/kaggle/input/dataset-vlsp2018/2-VLSP2018-SA-Hotel-dev.txt")
test_df = make_df("/kaggle/input/dataset-vlsp2018/3-VLSP2018-SA-Hotel-test.txt")

train_df

,comment,aspect,sentiment
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,"[HOTEL#DESIGN&FEATURES, HOTEL#GENERAL]","[positive, negative]"
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...",[LOCATION#GENERAL],[positive]
2,"Phục vụ, view đẹp, vị trí",[ROOMS#DESIGN&FEATURES],[positive]
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng","[HOTEL#COMFORT, HOTEL#CLEANLINESS, SERVICE#GEN...","[positive, positive, positive]"
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,"[LOCATION#GENERAL, FACILITIES#DESIGN&FEATURES,...","[positive, positive, positive]"
...,...,...,...
2995,Đối diện thung lũng Một kỳ nghỉ tuyệt vời ở Sa...,"[HOTEL#COMFORT, SERVICE#GENERAL, FACILITIES#DE...","[positive, positive, negative]"
2996,Cảm nhận của cá nhân về Sapa View: Phòng ốc đẹ...,"[ROOMS#DESIGN&FEATURES, ROOMS#CLEANLINESS, SER...","[positive, positive, positive, negative]"
2997,"Xin chào Chudu24,Xin cám ơn dịch vụ booking củ...","[HOTEL#COMFORT, ROOMS#CLEANLINESS, ROOMS#COMFO...","[positive, positive, positive, positive, neutr..."
2998,"Đầu tiên về ưu điểm:- Phòng sạch sẽ, nội thất ...","[ROOMS#CLEANLINESS, ROOM_AMENITIES#DESIGN&FEAT...","[positive, positive, positive, negative, negat..."


In [64]:
pip install vncorenlp

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [65]:
from vncorenlp import VnCoreNLP
word_segmenter = VnCoreNLP("/kaggle/input/vncorenlp/vncorenlp/VnCoreNLP-1.2.jar", annotators="wseg", quiet=False) 

def preprocessing(text):
    return ' '.join(sum(word_segmenter.tokenize(text), [])) # Flatten the list of words

text = "Đại học Bách Khoa Hà Nội."

print(preprocessing(text))

2024-12-09 03.59.12 INFO VnCoreNLPServer - Using annotators: wseg
2024-12-09 03:59:12 INFO  WordSegmenter:24 - Loading Word Segmentation model
2024-12-09 03.59.13 INFO VnCoreNLPServer - VnCoreNLPServer is listening on http://127.0.0.1:55811
2024-12-09 03.59.13 INFO log - Logging initialized @661ms to org.eclipse.jetty.util.log.Slf4jLog
2024-12-09 03.59.13 INFO EmbeddedJettyServer - == Spark has ignited ...
2024-12-09 03.59.13 INFO EmbeddedJettyServer - >> Listening on 127.0.0.1:55811
2024-12-09 03.59.13 INFO Server - jetty-9.4.z-SNAPSHOT, build timestamp: 2017-11-21T21:27:37Z, git hash: 82b8fb23f757335bb3329d540ce37a2a2615f0a8
2024-12-09 03.59.13 INFO session - DefaultSessionIdManager workerName=node0
2024-12-09 03.59.13 INFO session - No SessionScavenger set, using defaults
2024-12-09 03.59.13 INFO session - Scavenging every 660000ms
2024-12-09 03.59.13 INFO AbstractConnector - Started ServerConnector@404d86a4{HTTP/1.1,[http/1.1]}{127.0.0.1:55811}
2024-12-09 03.59.13 INFO Server - Sta

In [66]:
train_df['processed_comment'] = train_df['comment'].map(preprocessing)
val_df['processed_comment'] = val_df['comment'].map(preprocessing)
test_df['processed_comment'] = test_df['comment'].map(preprocessing)


In [67]:
train_df['processed_comment'][:10]

0    Rộng_rãi KS mới nhưng rất vắng . Các dịch_vụ c...
1    Địa_điểm thuận_tiện , trong vòng bán_kính 1,5 ...
2                          Phục_vụ , view đẹp , vị_trí
3               thuận_tiện , sạch_sẽ , vui_vẻ hài_lòng
4    Vị_trí đẹp ; Có quán bar view đẹp ; Nhân_viên ...
5    - Co view huong Ho tay - sach se-nhan vien tan...
6    Phòng_ốc sạch , giường thoải_mái , nhân_viên t...
7    gần Hồ Tây , view nhìn ra hồ lãng_mạn , đi bộ ...
8    Hình_thức không_thể quyết_định nội dung.H & at...
9    Tôi ở đây lần này là lần thứ 4 . Khi nhận phòn...
Name: processed_comment, dtype: object

In [68]:
#Take unique aspect#aspect

listAspects = train_df['aspect'].values.tolist()
uniqueAspect = set()

for aspects in listAspects:
    for aspect in aspects:
        uniqueAspect.add(aspect)

uniqueAspects = list(uniqueAspect)
print(f"size of Aspects : {len(uniqueAspects)}")
uniqueAspects

size of Aspects : 34


['ROOMS#GENERAL',
 'HOTEL#QUALITY',
 'SERVICE#GENERAL',
 'FOOD&DRINKS#MISCELLANEOUS',
 'ROOM_AMENITIES#PRICES',
 'FACILITIES#QUALITY',
 'ROOMS#QUALITY',
 'ROOM_AMENITIES#MISCELLANEOUS',
 'HOTEL#GENERAL',
 'ROOM_AMENITIES#CLEANLINESS',
 'FACILITIES#CLEANLINESS',
 'FOOD&DRINKS#PRICES',
 'ROOM_AMENITIES#QUALITY',
 'HOTEL#COMFORT',
 'ROOM_AMENITIES#GENERAL',
 'FACILITIES#MISCELLANEOUS',
 'LOCATION#GENERAL',
 'ROOMS#COMFORT',
 'FACILITIES#DESIGN&FEATURES',
 'ROOM_AMENITIES#COMFORT',
 'ROOMS#PRICES',
 'ROOMS#DESIGN&FEATURES',
 'HOTEL#DESIGN&FEATURES',
 'FOOD&DRINKS#QUALITY',
 'FACILITIES#PRICES',
 'ROOMS#CLEANLINESS',
 'ROOM_AMENITIES#DESIGN&FEATURES',
 'HOTEL#MISCELLANEOUS',
 'FACILITIES#COMFORT',
 'ROOMS#MISCELLANEOUS',
 'HOTEL#CLEANLINESS',
 'FOOD&DRINKS#STYLE&OPTIONS',
 'FACILITIES#GENERAL',
 'HOTEL#PRICES']

In [69]:
le = LabelEncoder()
le.fit(uniqueAspects)
allAspects = le.classes_
print(allAspects)
le.transform(['FACILITIES#CLEANLINESS','FACILITIES#GENERAL'])

['FACILITIES#CLEANLINESS' 'FACILITIES#COMFORT'
 'FACILITIES#DESIGN&FEATURES' 'FACILITIES#GENERAL'
 'FACILITIES#MISCELLANEOUS' 'FACILITIES#PRICES' 'FACILITIES#QUALITY'
 'FOOD&DRINKS#MISCELLANEOUS' 'FOOD&DRINKS#PRICES' 'FOOD&DRINKS#QUALITY'
 'FOOD&DRINKS#STYLE&OPTIONS' 'HOTEL#CLEANLINESS' 'HOTEL#COMFORT'
 'HOTEL#DESIGN&FEATURES' 'HOTEL#GENERAL' 'HOTEL#MISCELLANEOUS'
 'HOTEL#PRICES' 'HOTEL#QUALITY' 'LOCATION#GENERAL' 'ROOMS#CLEANLINESS'
 'ROOMS#COMFORT' 'ROOMS#DESIGN&FEATURES' 'ROOMS#GENERAL'
 'ROOMS#MISCELLANEOUS' 'ROOMS#PRICES' 'ROOMS#QUALITY'
 'ROOM_AMENITIES#CLEANLINESS' 'ROOM_AMENITIES#COMFORT'
 'ROOM_AMENITIES#DESIGN&FEATURES' 'ROOM_AMENITIES#GENERAL'
 'ROOM_AMENITIES#MISCELLANEOUS' 'ROOM_AMENITIES#PRICES'
 'ROOM_AMENITIES#QUALITY' 'SERVICE#GENERAL']


array([0, 3])

In [70]:
# One-hot encode data
def one_hot_encode(aspects, sentiments):
    aspect_dict = {aspect: [0, 0, 0, 1] for aspect in allAspects}
    for aspect, sentiment in zip(aspects, sentiments):
        if aspect in allAspects:
            if sentiment == "positive":
                aspect_dict[aspect] = [1, 0, 0, 0]
            elif sentiment == "neutral":
                aspect_dict[aspect] = [0, 1, 0, 0]
            elif sentiment == "negative":
                aspect_dict[aspect] = [0, 0, 1, 0]
    return np.array(list(aspect_dict.values()))

def make_data_for_training(data):
    comments = data['processed_comment'].values
    labels = np.array([one_hot_encode(aspects, sentiments) for aspects, sentiments in zip(data["aspect"], data["sentiment"])])
    return comments,labels

In [72]:
train_comment,train_label = make_data_for_training(train_df)
val_comment,val_label = make_data_for_training(val_df)
test_comment,test_label = make_data_for_training(test_df)

In [74]:
#Use tokenizer and model after fine-tuned

tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/absa-fine-tuned-model/model")
model = AutoModelForSequenceClassification.from_pretrained(
    "/kaggle/input/absa-fine-tuned-model/model"
)

class CustomDataset(Dataset):
        def __init__(self, comments, labels, tokenizer, max_length):
            self.comments = comments
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length
            print(f"Number of comments: {len(self.comments)}")
            print(f"Number of labels: {len(self.labels)}")

        def __len__(self):
            return len(self.comments)

        def __getitem__(self, idx):
            try:
                comment = self.comments[idx]
                label = self.labels[idx]
    
                # Tokenize the comment using the tokenizer
                encoding = self.tokenizer(
                    comment,
                    truncation=True,
                    padding='max_length',
                    max_length=self.max_length,
                    return_tensors='pt'
                )

                
                input_ids = encoding['input_ids'].squeeze(0)
                attention_mask = encoding['attention_mask'].squeeze(0)
    
                # Flatten the label to match the shape of output ((34,4) to 136)
                label_flat = torch.tensor(label, dtype=torch.float).flatten()
                return {
                    'input_ids': input_ids,
                    'attention_mask': attention_mask,
                    'labels': label_flat
                }
            except Exception as e:
                print(f"Error encountered at idx: {idx}")
                label = self.labels[idx]
                print(label)
                print(f"Comment: {self.comments[idx] if idx < len(self.comments) else 'N/A'}")
                print(f"Label: {self.labels[idx] if idx < len(self.labels) else 'N/A'}")
                raise e

# Create your custom dataset
train_dataset = CustomDataset(train_comment, train_label, tokenizer, max_length=256)
val_dataset = CustomDataset(val_comment, val_label, tokenizer, max_length=256)
test_dataset = CustomDataset(test_comment, test_label, tokenizer, max_length=256)


Number of comments: 3000
Number of labels: 3000
Number of comments: 2000
Number of labels: 2000
Number of comments: 600
Number of labels: 600


# 2. Fine-tune model


In [15]:
# from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# # Define total_steps
# total_steps = len(train_comment)

# # Define TrainingArguments
# training_args = TrainingArguments(
#     output_dir="./results/output",
#     learning_rate=1e-4,  # Initial learning rate
#     num_train_epochs=10,  # Max number of epochs
#     per_device_train_batch_size=16,
#     eval_strategy="steps",  # Evaluate every few steps
#     eval_steps=500,
#     logging_steps=500,
#     save_steps=1000,
#     warmup_steps=int(total_steps * 0.15),  # Warmup phase for learning rate
#     weight_decay=0.01,  # AdamW weight decay
#     save_total_limit=3,  # Keep only 3 model checkpoints
#     load_best_model_at_end=True,  # Restore best weights
#     metric_for_best_model="eval_loss",  # Monitor validation loss
#     greater_is_better=False,  # Lower validation loss is better
#     lr_scheduler_type="cosine",  # Use cosine scheduler here
#     report_to="none"
# )

# # Define EarlyStoppingCallback
# early_stopping = EarlyStoppingCallback(
#     early_stopping_patience=3  # Stop training if no improvement in 3 evaluation rounds
# )

# # Define Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     callbacks=[early_stopping]
# )

# # Start training
# trainer.train()

Step,Training Loss,Validation Loss
500,0.072100,0.086653
1000,0.061600,0.081378
1500,0.046800,0.079447


TrainOutput(global_step=1880, training_loss=0.056388245237634534, metrics={'train_runtime': 807.1283, 'train_samples_per_second': 37.169, 'train_steps_per_second': 2.329, 'total_flos': 3951414190080000.0, 'train_loss': 0.056388245237634534, 'epoch': 10.0})

In [16]:
# output = "/kaggle/working/savedmodel"
# model.save_pretrained(output)
# tokenizer.save_pretrained(output)

('/kaggle/working/savedmodel/tokenizer_config.json',
 '/kaggle/working/savedmodel/special_tokens_map.json',
 '/kaggle/working/savedmodel/vocab.txt',
 '/kaggle/working/savedmodel/bpe.codes',
 '/kaggle/working/savedmodel/added_tokens.json')

# 3. Test with test_dataset


In [75]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/absa-fine-tuned-model/model")
model = AutoModelForSequenceClassification.from_pretrained(
    "/kaggle/input/absa-fine-tuned-model/model"
)

In [76]:
test_dataset = CustomDataset(
    comments=test_comment,  # List of test comments
    labels=test_label,      # List of test labels
    tokenizer=tokenizer,     # Use the same tokenizer
    max_length=256           # Same max_length used during training
)

Number of comments: 600
Number of labels: 600


In [77]:
trainer = Trainer(model=model, tokenizer=tokenizer)

# Get predictions
predictions = trainer.predict(test_dataset)
predicted_logits = predictions.predictions  # Raw logits

predicted_logits

array([[ -6.5264163,  -9.010843 ,  -4.8486423, ...,  -1.6419296,
          2.3865283,  -4.6474495],
       [ -6.293882 ,  -9.900717 ,  -5.438618 , ...,  -4.65633  ,
         -4.53457  ,  -4.406285 ],
       [ -7.625139 , -12.377776 ,  -7.2884483, ...,  -5.316563 ,
         -4.3896008,   3.1430576],
       ...,
       [ -7.321956 , -12.25127  ,  -6.807439 , ...,  -5.1848507,
         -4.9976363,  -5.5306344],
       [ -5.119636 , -10.319636 ,  -5.6162066, ...,  -5.3497972,
         -4.795535 ,  -5.545847 ],
       [ -4.6908836, -10.440037 ,  -7.113595 , ...,  -5.4093223,
         -4.5841403,   2.465668 ]], dtype=float32)

In [78]:
# Reshape the logits
reshaped_logits = np.reshape(predicted_logits, (len(test_dataset), 34, 4))

# Compute the softmax along the last axis (4 categories)
softmax_logits = np.exp(reshaped_logits) / np.sum(np.exp(reshaped_logits), axis=2, keepdims=True)

print(softmax_logits.shape)
print(softmax_logits[0])  # Softmax probabilities for the first sample

(600, 34, 4)
[[6.81186930e-06 5.67928225e-07 3.64681982e-05 9.99956131e-01]
 [1.72593227e-05 1.06514585e-06 4.25054932e-05 9.99939144e-01]
 [2.35677794e-06 8.92751723e-06 1.16460855e-04 9.99872267e-01]
 [8.27781041e-05 1.84891651e-05 3.38350714e-04 9.99560356e-01]
 [4.05917126e-06 5.22186738e-09 1.02780014e-05 9.99985635e-01]
 [4.78671609e-06 1.48906878e-07 8.38063643e-05 9.99911189e-01]
 [2.70290348e-05 9.07442825e-07 2.40372799e-04 9.99731719e-01]
 [4.13594353e-06 7.51817936e-07 5.45297780e-06 9.99989688e-01]
 [4.00676754e-08 1.09882627e-08 1.69476806e-07 9.99999762e-01]
 [3.89746162e-07 2.72878742e-06 4.54920701e-06 9.99992311e-01]
 [3.49492922e-07 2.85579745e-06 5.32488366e-06 9.99991417e-01]
 [2.02182904e-02 7.82989431e-04 2.61485353e-02 9.52850223e-01]
 [4.28115338e-04 6.21462008e-04 3.23881134e-02 9.66562271e-01]
 [7.61453193e-05 9.53576018e-05 1.59596407e-03 9.98232543e-01]
 [1.01646883e-05 3.79079807e-04 1.23889733e-03 9.98371899e-01]
 [1.75471278e-03 3.16101790e-07 2.39426922

In [79]:
max_logits = np.argmax(softmax_logits, axis=2)  # Shape: (num_samples, 34)

def change_to_label(MAX_LOGIT):
    label = []
    for i,idx in enumerate(MAX_LOGIT):
        if idx == 0:
            label.append(f"{allAspects[i]}#positive")
        elif idx == 1:
            label.append(f"{allAspects[i]}#neutral")
        elif idx == 2:
            label.append(f"{allAspects[i]}#negative")
    return label

for i in range(0,20,1):
    label = change_to_label(max_logits[i])
    print(test_comment[i])
    print(f"\nPredicted labels: {label}\n")
    print("True labels:")
    for aspect,sentiment in zip(test_df['aspect'].iloc[i],test_df['sentiment'].iloc[i]):
        print(aspect+"#"+sentiment+" ")
    print("----------\n")

Ga giường không sạch , nhân_viên quên dọn phòng một ngày .

Predicted labels: ['SERVICE#GENERAL#negative']

True labels:
ROOM_AMENITIES#CLEANLINESS#negative 
SERVICE#GENERAL#negative 
----------

Nv nhiệt_tình , phòng ở sạch_sẽ , tiện_nghi , vị_trí khá thuận_tiện cho việc di_chuyển đến các địa_điểm ăn + chơi Phòng có gián

Predicted labels: ['LOCATION#GENERAL#positive', 'ROOMS#CLEANLINESS#positive', 'ROOMS#COMFORT#positive', 'SERVICE#GENERAL#positive']

True labels:
SERVICE#GENERAL#positive 
ROOMS#CLEANLINESS#neutral 
ROOMS#COMFORT#positive 
LOCATION#GENERAL#positive 
----------

Đi bộ ra biển gần , tiện đi_lại Phòng view biển nhưng cửa_sổ view biển khá bé

Predicted labels: ['LOCATION#GENERAL#positive', 'ROOMS#DESIGN&FEATURES#negative']

True labels:
LOCATION#GENERAL#positive 
ROOMS#GENERAL#positive 
ROOMS#DESIGN&FEATURES#neutral 
----------

Tất_cả mọi thứ đều sạch_sẽ , giường ngủ rất thoải_mái . Không có quạt_điện mà chỉ có_điều hoà nên có chút bất_tiện .

Predicted labels: ['

# 4. Evaluation with test_dataset


In [84]:
# Convert softmax probabilities to predicted classes
predicted_classes = np.argmax(softmax_logits, axis=2)  # Shape: (num_samples, 34)


# Convert true labels to actual classes
true_classes = np.argmax(test_label, axis=2)  # Shape: (num_samples, 34)

# Calculate per-sample accuracy
correct_predictions = (predicted_classes == true_classes)
accuracy_per_sample = np.mean(correct_predictions, axis=1)  # Accuracy per sample
overall_accuracy = np.mean(correct_predictions)  # Overall accuracy

print(f"Overall accuracy: {overall_accuracy:.4f}")

Overall accuracy: 0.9364


In [87]:
category_dict ={aspect:0 for aspect in allAspects}

for i in range(0,len(test_label),1):
    for j in range(0,34,1):
        if true_classes[i][j] == predicted_classes[i][j]:
            temp_category = le.inverse_transform([j])
            category_dict[temp_category[0]]+=1     
 
for aspect in allAspects:
    category_dict[aspect]/=600
    category_dict[aspect] = round(category_dict[aspect], 4)
    
assessment = pd.DataFrame([category_dict])
assessment

,FACILITIES#CLEANLINESS,FACILITIES#COMFORT,FACILITIES#DESIGN&FEATURES,FACILITIES#GENERAL,FACILITIES#MISCELLANEOUS,FACILITIES#PRICES,FACILITIES#QUALITY,FOOD&DRINKS#MISCELLANEOUS,FOOD&DRINKS#PRICES,FOOD&DRINKS#QUALITY,...,ROOMS#PRICES,ROOMS#QUALITY,ROOM_AMENITIES#CLEANLINESS,ROOM_AMENITIES#COMFORT,ROOM_AMENITIES#DESIGN&FEATURES,ROOM_AMENITIES#GENERAL,ROOM_AMENITIES#MISCELLANEOUS,ROOM_AMENITIES#PRICES,ROOM_AMENITIES#QUALITY,SERVICE#GENERAL
0,0.9883,0.9583,0.9033,0.9683,0.9867,0.9783,0.9183,0.995,0.985,0.9367,...,0.9633,0.985,0.925,0.89,0.8183,0.9817,0.995,0.9983,0.89,0.905


# 5. Input your comment and try


In [82]:
# Ensure model is on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

while True:
    print("Fill your comment: ")
    text = input()
    if text == "":
        print("Exiting...")
        break
        
    segmented_text = preprocessing(text)
    
    # Tokenize the input text
    tokenized_text = tokenizer(
        segmented_text,
        max_length=256,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    # Move tokenized_text to the same device as the model
    tokenized_text = {key: value.to(device) for key, value in tokenized_text.items()}

    # Perform prediction
    with torch.no_grad():
        logits = model(**tokenized_text).logits

    # Process logits and output predictions (e.g., softmax)
    softmax = torch.nn.Softmax(dim=-1)
    reshaped_logits = torch.reshape(logits,(1,34,4))
    probabilities = softmax(reshaped_logits)  # Shape: (1, 34)
    predicted_classes = torch.argmax(probabilities, dim=-1)  # Shape: (1, 34)
    predicted_classes = predicted_classes.cpu().numpy()
    label = change_to_label(predicted_classes[0])
    print(f"\nPredicted labels: {label}\n")
    print("----------\n")

Fill your comment: 


 Lễ tân thân thiện, có thang máy, vị trí ks thuận tiện, view thành phố rất đẹp. Phòng sạch nhưng hơi nhỏ & thiếu bình đun siêu tốc. Sẽ quay lại & giới thiệu bạn bè



Predicted labels: ['HOTEL#DESIGN&FEATURES#positive', 'HOTEL#GENERAL#positive', 'LOCATION#GENERAL#positive', 'ROOMS#CLEANLINESS#positive', 'ROOMS#DESIGN&FEATURES#negative', 'SERVICE#GENERAL#positive']

----------

Fill your comment: 


Exiting...
